In [1]:
!pip install bert-score

In [2]:
import json
import numpy as np

from bert_score import score

In [3]:
file_name = 'base_paligemma_uamemes_predicted_answers.json'

with open(file_name, 'r') as file:
    results = json.load(file)

In [4]:
ground_truth = list(map(lambda entry: entry['ground-truth-answer'], results))
predicted = list(map(lambda entry: entry['answer'], results))

In [5]:
precisions, recalls, f1_scores = score(predicted, ground_truth, lang="uk", verbose=True)

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 2.54 seconds, 114.40 sentences/sec


In [6]:
precisions = list(map(lambda score: score.item(), precisions))
recalls = list(map(lambda score: score.item(), recalls))
f1_scores = list(map(lambda score: score.item(), f1_scores))

In [7]:
for i, (precision, recall, f1) in enumerate(zip(precisions, recalls, f1_scores)):
    metric = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    results[i]['metric'] = metric

In [8]:
top_10_indices = np.argsort(f1_scores)[-10:][::-1]
np.array(results)[top_10_indices]

array([{'ground-truth-answer': 'Відступ російської армії з Харківщини', 'answer': 'Відхід російських військ з України.', 'metric': {'precision': 0.8766322135925293, 'recall': 0.8770008683204651, 'f1': 0.8768165707588196}},
       {'ground-truth-answer': 'Вибух на Кримському мосту', 'answer': 'Вибух на мосту АТО.', 'metric': {'precision': 0.8383924961090088, 'recall': 0.8422766327857971, 'f1': 0.8403301239013672}},
       {'ground-truth-answer': 'Виявилося, що російські військові копали окопи в Чорнобильській зоні', 'answer': 'Російські військові входить у Чернобиль.', 'metric': {'precision': 0.8351117372512817, 'recall': 0.7472822666168213, 'f1': 0.7887595891952515}},
       {'ground-truth-answer': 'В Україні відзначили День військової розвідки', 'answer': 'З Днем Военної розвідки.', 'metric': {'precision': 0.8099087476730347, 'recall': 0.7515135407447815, 'f1': 0.7796192169189453}},
       {'ground-truth-answer': 'Чорнобиль', 'answer': 'Російські військові входить у Чернобиль.', 'metr

In [9]:
worst_10_indices = np.argsort(f1_scores)[:10]
np.array(results)[worst_10_indices]

array([{'ground-truth-answer': 'Київська обл;Київ', 'answer': 'На зображенні зображено мем про подію «Кіт, який помер, коли його вдарили по голові ракеткою».', 'metric': {'precision': 0.4970638155937195, 'recall': 0.540182888507843, 'f1': 0.5177270770072937}},
       {'ground-truth-answer': 'Фінляндія', 'answer': 'У цьому мемі зображено зустріч Володимира Путіна з президентом України Володимиром Зеленкою та лідером NATO Джоном Керрі.', 'metric': {'precision': 0.5007182359695435, 'recall': 0.5395541787147522, 'f1': 0.5194112658500671}},
       {'ground-truth-answer': 'НАТО', 'answer': '75-річчя незалежності України.', 'metric': {'precision': 0.47855091094970703, 'recall': 0.5827016234397888, 'f1': 0.525515615940094}},
       {'ground-truth-answer': 'F-16', 'answer': 'Річка Різні та відкриття Східного порту в Києві.', 'metric': {'precision': 0.5282252430915833, 'recall': 0.5229735970497131, 'f1': 0.5255863070487976}},
       {'ground-truth-answer': 'Київ', 'answer': 'На зображенні зображ

In [10]:
result_filename = file_name.split('.')[0] + 'with_metrics.' + file_name.split('.')[1]

with open(result_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)